# Programmatic Replacements for NICMOS Units Conversion Form

This notebook illustrates programmatic ways to perform unit conversions as provided by [NICMOS Units Conversion Form](http://www.stsci.edu/hst/nicmos/tools/conversion_form.html). The examples here are not exhaustive and will never be. They are meant to give users enough basic knowledge to then calculate what they really want.

## Imports
- All the examples require `astropy.units`. See `Units` and `Quantities` in Astropy for more information.
- Some examples require the package `snyphot`.
- Some examples require the `math` library. 

In [1]:
from astropy import units as u

from synphot import SourceSpectrum
from synphot import units as syn_u
from synphot.models import BlackBodyNorm1D

import math

## Example 1: Simple Jy to AB mag

```
INPUT FROM THE FORM IS 
Input units = Jy             
Output units = AB magnitude   
Input flux =    1.00000E-13 Jy             
Temperature of the blackbody =    5500.00
INPUT wavelength =     1.00000 micron
OUTPUT wavelength =     1.00000 micron
AB magnitude   = 41.43
```

In plain English, user wants to know what is 1e-13 Jy converted to AB magnitude at 1 micron. Blackbody is not needed for this conversion.

In [2]:
input_flux_ex1 = 1e-13 * u.Jy
input_wave_ex1 = 1 * u.micron
abmag_ex1 = input_flux_ex1.to(u.ABmag, u.spectral_density(input_wave_ex1))
print('{:.2f}'.format(abmag_ex1))

41.40 mag(AB)


## Example 2: PHOTLAM to FNU for Blackbody

```
INPUT FROM THE FORM IS 
Input units = photons/cm2/s/A
Output units = erg/cm2/s/Hz   
Input flux =    1.00000 photons/cm2/s/A
Temperature of the blackbody =    5500.00
INPUT wavelength =    0.500000 micron
OUTPUT wavelength =    0.600000 micron
Flux=4.62E-23 erg/cm2/s/Hz   
```

This example is more complicated in that it requires assumption of a blackbody and has different input and output units.

First, we create a source spectrum with a blackbody model with the given temperature of 5500 K.

In [3]:
bb_ex2 = SourceSpectrum(BlackBodyNorm1D, temperature=5500*u.K)

Then, we calculate the normalization factor required for the blackbody to have the given input flux at the given input wavelength.

In [4]:
input_flux_ex2 = 1 * syn_u.PHOTLAM
input_wave_ex2 = 0.5 * u.micron
factor_ex2 = input_flux_ex2 / bb_ex2(input_wave_ex2)

We apply this factor to our blackbody source.

In [5]:
input_ex2 = bb_ex2 * factor_ex2

Finally, we calculate the desired flux in given output unit and wavelength.

In [6]:
output_wave_ex2 = 0.6 * u.micron
flux_ex2 = input_ex2(output_wave_ex2, flux_unit=syn_u.FNU)
print('{:.2e}'.format(flux_ex2))

4.62e-23 FNU


## Example 3: W/m2/Hz to I-mag for Power-Law


```
INPUT FROM THE FORM IS 
Input units = W/m2/Hz        
Output units = magnitude I    
Input flux =    1.00000E-23 W/m2/Hz        
Index of the power-law spectrum as a function of frequency =   0.250000
INPUT wavelength =     1.00000 micron
OUTPUT wavelength =    0.900000 micron
I =  0.85
```

First, we define the input flux and wavelength.

In [7]:
input_flux_ex3 = 1e-23 * (u.W / (u.m * u.m * u.Hz))
input_wave_ex3 = 1 * u.micron

Then, we define a power-law function.

In [8]:
def powerlaw_ex3(nu):
    """F(nu)=nu**(spectral index)
    
    nu is a Quantity.
    """
    spectral_index = 0.25
    return (nu.to(u.Hz, u.spectral()) ** spectral_index).value

We use this power-law and a normalization factor based on input flux and wavelength to calculate output flux at output wavelength in input flux unit.

In [9]:
output_wave_ex3 = 0.9 * u.micron
flux_ex3 = powerlaw_ex3(output_wave_ex3) * input_flux_ex3 / powerlaw_ex3(input_wave_ex3) 

Finally, we convert the flux to *I* magnitude, as defined by the converter as:

```
magnitude I: zero-flux=2250 Jy; central wavelength=0.90 microns;
```

In [10]:
imag_zpt = 2250 * u.Jy
i = -2.5 * math.log10(flux_ex3 / imag_zpt)
print('{:.2f}'.format(i))

0.85
